In [ ]:
# sample.py
import ctypes
import os
import threading
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import math
import concurrent.futures

# Running model as a subprocess

In [ ]:
# Try to locate the .so file in the same directory as this file
_file = 'libeemodel.dylib'
_path = os.path.join(*(os.path.split(_file)[:-1] + (_file,)))
_mod = ctypes.cdll.LoadLibrary(_path)

In [ ]:
print(_mod)

In [ ]:
# parameters for the model
args = [b'0', b'NEW', b'/Users/wilfeli/Dropbox/ABM/SolarPanels/ABMIRISLab/Source/Examples/EEModel/eemodel.model']
args_C = (ctypes.c_char_p * len(args))()
args_C[:] = args
#alternative for setting args_C (not tested)
#args_C = (ctypes.c_char_p * len(args))(*args)
#and could use create_string_buffer("string") to create array of strings

In [ ]:
_mod.init_model.restype = ctypes.c_void_p
_mod.init_model.argtypes = [ctypes.c_int , ctypes.POINTER(ctypes.c_char_p)]
_mod.run_model.restype = ctypes.c_int
_mod.run_model.argtypes = [ctypes.c_void_p]
_mod.C_API_run_model_steps.restype = ctypes.c_int
_mod.C_API_run_model_steps.argtypes = [ctypes.c_void_p, ctypes.c_int]

class HUIBL(ctypes.Structure):
    _fields_ = []
    
_mod.C_API_estimate_profit.restype = ctypes.c_double
_mod.C_API_estimate_profit.argtypes = [ctypes.POINTER(HUIBL), ctypes.c_int , ctypes.c_double]
_mod.C_API_estimate_irr.restype = ctypes.c_double
_mod.C_API_estimate_irr.argtypes = [ctypes.POINTER(HUIBL), ctypes.c_int , ctypes.c_double]
_mod.C_API_estimate_demand_sei_params.restype = ctypes.c_double
_mod.C_API_estimate_demand_sei_params.argtypes = [ctypes.POINTER(HUIBL), ctypes.c_int, \
                                       ctypes.c_double, ctypes.c_int, ctypes.POINTER(ctypes.c_double)]
_mod.C_API_estimate_demand_w_params.restype = ctypes.c_double
_mod.C_API_estimate_demand_w_params.argtypes = [ctypes.POINTER(HUIBL), ctypes.c_double]
_mod.C_API_estimate_irr_params.restype = ctypes.c_double
_mod.C_API_estimate_irr_params.argtypes = [ctypes.POINTER(HUIBL), ctypes.c_int , ctypes.c_double]


In [ ]:
#initializing model and getting ui pointer from initialization
huibl_void = _mod.init_model(len(args_C), args_C)

In [ ]:
print(huibl_void)

In [ ]:
huibl = ctypes.cast(huibl_void, ctypes.POINTER(HUIBL))

In [ ]:
#running model after it is initialized
res2 = _mod.run_model(huibl_void)

In [ ]:
#wrapper function to run model to be used in concurrent
def run_model(huibl_void):
#    res = _mod.run_model_test( len(args_C_), args_C_)
    _mod.run_model(huibl_void)

In [ ]:
#wrapper function to run model for fixed number of steps to be used for testing
def run_model_step(huibl_void):
    _mod.C_API_run_model_steps(huibl_void, 1)

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(run_model, huibl_void)
#alternative that does not work is with threads
#t1 = threading.Thread(target=_mod.run_model, args = [res])
#t1.start()    
#t1.join()

In [ ]:
def estimate_profit(huibl, p):
    return _mod.C_API_estimate_profit(huibl, 0, p)

In [ ]:
def estimate_irr(huibl, p):
    return _mod.C_API_estimate_irr(huibl, 0, p)

In [ ]:
def estimate_irr(huibl, eff):
    return _mod.C_API_estimate_irr_params(huibl, 0, eff)

In [ ]:
def estimate_demand(huibl, irr):
    return _mod.C_API_estimate_demand_w_params(huibl, irr)

In [ ]:
#parameters for demand estimation
scale_factor = 0.666666666667
theta = np.array([-1.75, 1.5, 1.0, -1.0, 1.0]) * scale_factor

In [ ]:
theta_C = (ctypes.c_double * len(theta))()
theta_C[:] = theta

In [ ]:
def estimate_demand(huibl, p):
    return _mod.C_API_estimate_demand_sei_params(huibl, 0, p, len(theta_C), theta_C)

In [ ]:
def estimate_demand(huibl, p):
    return _mod.C_API_estimate_demand(huibl, 0, p)

In [ ]:
run_model_step(huibl_void)

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(run_model_step, huibl_void)

# Initialization parameters - visuals

## graph for expected time before next maintenance

In [ ]:
lambda_ = 2500
alpha = 100
y = np.zeros(100)
x = np.zeros(100)
i = 0


for step in np.linspace(0.01, 25, num=100):
    x[i] = step
#    y[i] = lambda_ * math.pow((1 - x[i]), -(1/alpha)) - 1
    #cdf
    y[i] = 1 - math.pow(1 + x[i]/lambda_, - alpha)
    i = i + 1
#draw graph
plt.plot(x, y)

## Graph for expected share of sales

## Estimate scale factor

In [ ]:
theta = np.array([-0.005, 1.25, 0.005, -0.5, -0.005])

#assume equal reputation, equal high irr as priors 
irr_i = 0.1
rep = 1
rep_i = 1
irr = irr_i
X = np.array([1, irr, rep, irr_i, rep_i])


#split market equally given number of agents
N = 3

sum = np.dot(X, theta)
print(sum)

scale_factor = 1/sum/N
print (scale_factor)

theta = theta * scale_factor

In [ ]:
-0.1*1+2.0*0.1+0.05*1-1.5*0.1-0.05*1

## Draw demand share for the range of irrs

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
for step in np.linspace(0.01, 0.4, num=100):
    x[i] = step
    X[1] = x[i]
    y[i] = np.dot(X, theta)
    i = i + 1
#draw graph
plt.plot(x, y)

## Draw irr for the range of efficiencies

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
for step in np.linspace(0.15, 0.50, num=100):
    x[i] = step
    y[i] = estimate_irr(huibl, x[i])
    i = i + 1
#draw graph
plt.plot(x, y)

## Draw demand share as a function of reasonable price range (Python estimation)

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
for step in np.linspace(2, 10.0, num=100):
    x[i] = step
    X[1] = estimate_irr(huibl, x[i])
    y[i] = np.dot(X, theta)
    i = i + 1
#draw graph
plt.plot(x, y)

# Graph for expected profit given price

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
for step in np.linspace(0.01, 10.0, num=100):
    x[i] = step
    y[i] = estimate_profit(huibl, x[i])
    i = i + 1
#draw graph
plt.plot(x, y)


# Graph for expected irr given price

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
for step in np.linspace(2, 10.0, num=100):
    x[i] = step
    y[i] = estimate_irr(huibl, x[i])
    i = i + 1
#draw graph
plt.plot(x, y)


# Graph for expected demand given price, C++ code

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
for step in np.linspace(2, 10.0, num=100):
    x[i] = step
##    print(estimate_irr(huibl, x[i]))
    y[i] = estimate_demand(huibl, x[i])
    i = i + 1
#draw graph
plt.plot(x, y)

## Graph for realized demand given irr, C++ code

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
for step in np.linspace(0.01, 0.99, num=100):
    x[i] = step
    y[i] = estimate_demand(huibl, x[i])
    i = i + 1
#draw graph
plt.plot(x, y)

In [ ]:
theta_1 = np.array([-1.75, 1.5, 1.0, -1.0, 1.0])
theta = np.array([-1.1, 1.065, 0.7, -0.6, 0.7])
X = np.array([1, 1, 1/(2.1 - 1)+0.25, 1, 0.95])
print(np.dot(X, theta))
print(np.dot(X, theta_1))

## Inverse-Gamma

In [ ]:
import scipy as sp
import scipy.special

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
beta = 1
alpha = 3
for step in np.linspace(0.01, 10.0, num=100):
    x[i] = step
    y[i] = math.pow(beta, alpha) * math.pow(x[i], -alpha-1)*math.exp(-beta/x[i])/sp.special.gamma(alpha)
    i = i + 1
#draw graph
plt.plot(x, y)

In [ ]:
THETA_params = [1, 3, 0.02]
THETA_decision = [0, 0]
THETA_decision[0] = 1/THETA_params[1] * math.pow((1 + THETA_params[0]/THETA_params[1] * 50000 / 1000),-1/(THETA_params[0] + 1))
print(THETA_decision[0])
THETA_decision[1] = THETA_params[2]

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
a = 0.5
for step in np.linspace(30000, 100000, num=100):
    x[i] = step
    y[i] = 1/THETA_params[1] * math.pow((1 + THETA_params[0]/THETA_params[1] * x[i] / 1000),-1/(THETA_params[0] + 1))
    i = i + 1
#draw graph
plt.plot(x, y)

In [ ]:
y = np.zeros(100)
x = np.zeros(100)
i = 0
a = 0.5
for step in np.linspace(0.01, 0.1, num=100):
    x[i] = step
    y[i] = (1/(1+math.exp(-((x[i] + 0.15)*a - THETA_decision[0])/THETA_decision[1])))
    i = i + 1
#draw graph
plt.plot(x, y)

In [ ]:
# Model for the market

In [ ]:
#assume that California is half of the market 
#And SolarCity is half of the regional market 
#cost per watt marketing is 0.48
#total amount installed

In [ ]:
marketing_costs = 227*1000*0.48*1000
marketing_costs

In [ ]:
w = 20

In [ ]:
marketing_hours = marketing_costs/w
marketing_hours

In [ ]:
marketing_hours/2400

In [ ]:
#total number of installations
227*1000/6.7

In [ ]:
marketing_costs = 109000000

In [ ]:
#Model for california
#number of installations = 
N_installations_ca = 30000

In [ ]:
#GA costs from SolarCity financial report
g_a_costs = 86097000
g_a_hours = g_a_costs/w
g_a_hours

In [ ]:
#solar panel output
STC_panel = 0.26
system_size = 6
N_panels = system_size/STC_panel
N_panels

In [ ]:
#additional costs
#they state installation costs being $2 and substract production costs of 0.5 per watt
labor_hours = system_size * 1000 * (2 - 0.5)/w
#total labor hours in my model need to sum up to the estimate of labor hours for solarcity
labor_hours

In [ ]:
#Assume for california yearly market size as 30000 (very optimistic) 
#scale fixed costs accordingly - basically Solarcity is US - which is 50/50 California 
#and the rest and 50 percent of the market 
#they have roughly 30000 installations for US
#15000 - assume in CA and adjust for the market size - total will be 30000 - rought estimate

In [ ]:
1559*1000/1000000*0.25